In [1]:
## IMPORTACIÓN GENERAL DE LIBRERIAS Y VISUALIZACIÓN DE DATOS (matplotlib y seaborn)

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as DT
import warnings
import sys

%matplotlib inline
warnings.filterwarnings('ignore')
plt.style.use('default') 
sns.set(style="whitegrid") 
plt.rcParams['figure.figsize'] = (18, 18)

np.set_printoptions(threshold=sys.maxsize)

La Agencia Nacional de Estadísticas de Buenos Aires recolecta información de nacimientos cuando los padres registran a sus hijos en el registro civil a partir de una encuesta.

Esa información se encuentra disponible para su análisis en un csv con el siguiente formato (dia_nacimiento, mes_nacimiento, anio_nacimiento, peso_al_nacer, longitud_al_nacer, id_hospital, tipo_parto), donde el tipo de parto 1 es natural y 2 es cesárea.

Por otro lado la agencia cuenta con información histórica de los hospitales en otro csv con siguiente formato (id_hospital, dirección, promedio_nacimientos_mensual).

Se pide usar Pandas para:

Calcular la cantidad de nacimientos para cada uno de los hospitales para el mes de Octubre de 2017 e indicar aquellos hospitales que superan el promedio de nacimientos mensuales.

Comparando el mes de Octubre de 2017 indicar programáticamente si se incremento el % de cesáreas con respecto a ese mes del año 2016.

In [3]:
#Leemos el CSV
nacimientos = pd.read_csv('data/2017C2_2_nacimientos.csv')
historico = pd.read_csv('data/2017C2_2_historico.csv')

In [4]:
nacimientos.head()

,dia_nacimiento,mes_nacimiento,anio_nacimiento,peso_al_nacer,longitud_al_nacer,id_hospital,tipo_parto
0,17,8,2017,3,30,1,1
1,17,10,2017,3,30,2,2
2,21,10,2017,3,30,3,1
3,8,10,2017,2,30,2,2
4,9,10,2017,3,30,2,2


In [5]:
historico.head()

,id_hospital,dirección,promedio_nacimientos_mensual
0,1,Beruti,1
1,2,Juan B Justo,1
2,3,Santa Fe,1


In [6]:
nac_oct = nacimientos[(nacimientos['mes_nacimiento']==10)&(nacimientos['anio_nacimiento']==2017)]

In [24]:
cant_nac_hosp = nac_oct.groupby('id_hospital')[['tipo_parto']].count().rename(columns={"tipo_parto": "cant"}).reset_index()
cant_nac_hosp.head()

,id_hospital,cant
0,1,1
1,2,3
2,3,1


In [28]:
agrupado = historico.merge(cant_nac_hosp,how = 'outer', on='id_hospital')
agrupado = agrupado[agrupado['cant']>agrupado['promedio_nacimientos_mensual']][['id_hospital','cant', 'promedio_nacimientos_mensual']]
agrupado.head()

,id_hospital,cant,promedio_nacimientos_mensual
1,2,3,1


In [31]:

nac_oct = nacimientos[nacimientos['mes_nacimiento'] == 10]
porc_cesarea = nac_oct.groupby('anio_nacimiento').tipo_parto.agg(lambda x: (x==2).mean())

if porc_cesarea[2017] > porc_cesarea[2016]: 
    print("Se incrementó el % de cesáreas")
else: 
    print("No se incrementó el % de cesáreas")

Se incrementó el % de cesáreas


# -----------------------------------------

# PARCIAL 2
Tenemos un dataframe con la información de distintas playlists armadas por usuarios con el formato (playlist, song_id, description).

A su vez, contamos con un dataframe de canciones que contiene (song_id, singer, year, lenght, genres). Se pide generar un programa en Pandas que indique para cada playlist cual es el cantante predominante (con mas canciones incluidas dentro de esa lista)

Link

In [32]:
playlist = pd.read_csv('data/2017C2_1_playlists.csv')
canciones = pd.read_csv('data/2017C2_1_canciones.csv')

In [35]:
playlist.head()

,playlist,song_id,description
0,Los Ochenta,1,Muy buena
1,Los Ochenta,2,Pesima
2,Los Ochenta,3,Re divertida!
3,Los Ochenta,4,Excelente
4,Los Noventa,5,Excelente


In [36]:
canciones.head()

,song_id,singer,year,length,genres
0,1,Firulo,1986,3m,rock
1,2,Firulo,1986,3m,rock
2,3,Pipi,1987,3m,rock
3,4,Kratos,1985,3m,rock
4,5,Pinoccio,1996,3m,rock


In [37]:
playlist.merge(canciones, how='left').groupby('playlist')['singer'].agg(lambda x:x.mode())

playlist
Los Noventa    Pinoccio
Los Ochenta      Firulo
Name: singer, dtype: object

# ----------------------------------------------

# PARCIAL 3
Un sitio de E books tiene información sobre los reviews que los usuarios hacen de sus libros en un DataFrame con formato (user_id, book_id, rating, timestamp).

Por otro lado tenemos información en otro DataFrame que bajamos de GoodReads: (book_id, book_name, avg_rating). Podemos suponer que los Ids de los libros son compatibles.

Se pide usar Python Pandas para:

Obtener un DataFrame que indique el TOP5 de Ebooks en el sitio de Ebooks. (Para este punto se puede ignorar el segundo DataFrame).

Obtener un DataFrame que indique qué libros tienen una diferencia de rating promedio mayor al 20% entre el sitio de Ebooks y GoodReads.

In [38]:
ebooks = pd.read_csv('data/2017C1_1_ebooks.csv')
goodreads = pd.read_csv('data/2017C1_1_goodreads.csv')

In [39]:
ebooks.head()

,user_id,book_id,rating,timestamp
0,1,1,4,2017-05-28 23:38:05
1,2,1,5,2017-05-28 23:38:06
2,1,2,1,2017-05-29 23:30:05
3,2,4,3,2017-05-29 23:32:05
4,3,3,2,2017-05-29 22:12:12


In [40]:
goodreads.head()

,book_id,book_name,avg_rating
0,1,To Kill a MockingBird,5
1,2,Don Quixote,4
2,3,Lazarillo de Tormes,4
3,4,1984,5
4,5,Fifty Shades of Grey,1


In [52]:
rating_ebooks = ebooks.groupby('book_id')[['rating']].mean()
rating_ebooks.nlargest(5, 'rating')

,rating
book_id,
1,4.5
6,4.0
7,4.0
8,4.0
4,3.5


In [59]:
agrupo = rating_ebooks.merge(goodreads, on='book_id')
agrupo.head()

,book_id,rating,book_name,avg_rating
0,1,4.500000,To Kill a MockingBird,5
1,2,2.666667,Don Quixote,4
2,3,1.500000,Lazarillo de Tormes,4
3,4,3.500000,1984,5
4,5,2.000000,Fifty Shades of Grey,1


In [67]:
agrupo = agrupo[(((agrupo['rating']/agrupo['avg_rating'])-1).abs())>0.2][['book_name']]
agrupo

,book_name
1,Don Quixote
2,Lazarillo de Tormes
3,1984
4,Fifty Shades of Grey


# PARCIAL 4
El GCPD (Gotham City Police Dept) recolecta la información de casos policiales que acontecen en Ciudad Gótica. Esta información se encuentra guardada en un dataframe con el siguiente formato: (fecha, id_caso, descripcion, estado_caso, categoria, latitud, longitud).

Los posibles estados que puede tener un caso son 1: caso abierto, 2: caso resuelto, 3: cerrado sin resolución.

Las fechas se encuentran en el formato YYYY-MM-DD. Por otro lado el comisionado Gordon guarda un registro detallado sobre en cuáles casos fue activada la batiseñal para pedir ayuda del vigilante, Batman. Esta información se encuentra en un Dataframe con el siguiente formato (id_caso, respuesta), siendo campo respuesta si la señal tuvo una respu esta positiva (1) o negativa (0) de parte de él.

El sector encargado de las estadísticas oficiales del GCPD quiere con esta información analizar las siguientes situaciones:

a) Tasa de resolución de casos de la fuerza policial por categoría de caso (considerando aquellos casos en los que no participó Batman).

b) Tasa de resolución de casos con la ayuda de Batman (considerando que aquellos casos en los que fue llamado con la batiseñal, participó en la resolución).

c) Indicar el mes del año pasado en el que Batman tuvo mayor participación en la investigación de casos.

In [80]:
gcpd = pd.read_csv('data/2018C1_GCPD.csv')
gordon = pd.read_csv('data/2018C1_gordon.csv')

In [81]:
gcpd.head()

,fecha,id_caso,descripcion,estado_caso,categoria,latitud,longitud
0,2017-06-21,1,Desc1,1,Asesinato,40.75793,-73.98551
1,2017-06-26,2,Desc2,1,Robo,41.75213,-72.98285
2,2017-06-19,3,Desc3,2,Fraude,39.37793,-73.18525
3,2017-06-16,4,Desc4,3,Asesinato,40.75724,-72.58912
4,2017-06-23,5,Desc5,2,Fraude,40.71193,-73.59812


In [82]:
gordon.head()

,id_caso,respuesta
0,1,1
1,2,1
2,3,1
3,6,0
4,7,1


In [83]:
sin_batman = gordon[gordon['respuesta']==0]

In [84]:
sin_batman = sin_batman.merge(gcpd, on='id_caso')

In [85]:
sin_batman.head(15)

,id_caso,respuesta,fecha,descripcion,estado_caso,categoria,latitud,longitud
0,6,0,2017-07-08,Desc6,2,Asesinato,40.76393,-73.51294
1,12,0,2017-07-10,Desc12,1,Asesinato,40.15793,-71.21502
2,13,0,2017-07-12,Desc13,2,Asesinato,39.77283,-74.92351
3,17,0,2017-07-09,Desc17,2,Asesinato,40.61923,-72.12512


In [86]:
agrupado_por_resolucion = sin_batman.groupby(['estado_caso', 'categoria'])[['latitud']].count().reset_index().rename(columns={'latitud':'cant'})
agrupado_por_resolucion.head()

,estado_caso,categoria,cant
0,1,Asesinato,1
1,2,Asesinato,3
